In [6]:
import tensorflow_hub as hub
import tensorflow as tf
import os
import PIL
from PIL import Image
import numpy as np

os.environ['TFHUB_DOWNLOAD_PROGRESS'] = "1"

physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Load test image

In [2]:
img_path = "../dataset_test/brad.jpg"

# Load labels

## ImageNet: 1001

In [50]:
with open("../dataset_test/ImageNetLabels.txt") as f:
    labels_imagenet = np.asarray(f.readlines())
    labels_imagenet = [x.replace("\n", "") for x in labels_imagenet]

## imagenet21k_wordnet_lemmas

In [26]:
with open("../dataset_test/imagenet21k_wordnet_lemmas.txt") as f:
    labels_imagenet21k = np.asarray(f.readlines())
    labels_imagenet21k = [x.replace("\n", "") for x in labels_imagenet21k]

# Test Models

**Path to downloaded models:** /tmp/tfhub_modules/

## mobilenet_v2

In [73]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", output_shape=[1001])
])
m.build([None, 224, 224, 3])  # Batch input shape.

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
# Transform the image
#image = tf.keras.preprocessing.image.load_img(img_path)
image_pil = Image.open(img_path)
image = np.array(image_pil)
image = tf.image.resize(image, [224, 224])
image = normalization_layer(image)
image = tf.expand_dims(image, axis=0)
# Predict
prediction = m(image).numpy()[0]
# Map with labels
labels_mapped = np.column_stack([labels_imagenet, prediction])
labels_mapped_high = [[x, y] for (x, y) in labels_mapped if np.float(y) > 5]
# 
labels_mapped_high

FileNotFoundError: [Errno 2] No such file or directory: 'elephant.jpg'

In [54]:
prediction = m(image).numpy()

(1, 1001)

In [57]:
prediction[0]

array([ 0.4343094 , -1.6993018 , -0.37546688, ..., -1.886097  ,
       -2.4636226 ,  0.83712065], dtype=float32)

In [68]:
from keras.applications import imagenet_utils
P = imagenet_utils.decode_predictions(prediction[:,:1000], 10)

In [69]:
P

[[('n03417042', 'garbage_truck', 8.71818),
  ('n04355338', 'sundial', 5.782742),
  ('n02509815', 'lesser_panda', 5.212958),
  ('n04482393', 'tricycle', 5.083147),
  ('n02441942', 'weasel', 4.8728623),
  ('n03047690', 'clog', 4.8712473),
  ('n02504458', 'African_elephant', 4.8337836),
  ('n02111500', 'Great_Pyrenees', 4.308421),
  ('n02108915', 'French_bulldog', 3.9751885),
  ('n02113978', 'Mexican_hairless', 3.8408327)]]

In [80]:
model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=True)
image_pil = Image.open(img_path)
image = np.array(image_pil)
image = tf.image.resize(image, [224, 224])

x = np.expand_dims(np.copy(image), axis=0)
x = tf.keras.applications.vgg16.preprocess_input(x)

features = model.predict(x)


array([[5.55101860e-06, 1.28235104e-06, 6.18547256e-06, 5.16149657e-06,
        1.72082873e-04, 4.08447049e-05, 6.63919927e-05, 4.45081678e-04,
        4.51911983e-05, 9.60685725e-07, 2.58864333e-07, 5.51524920e-07,
        3.18804359e-06, 7.96642269e-07, 1.27646024e-06, 1.11191275e-06,
        9.98873134e-07, 2.16565536e-06, 6.40355483e-06, 1.58590717e-06,
        2.79125925e-06, 5.05311255e-06, 4.49886429e-05, 1.84312212e-05,
        1.26166117e-06, 2.77181243e-06, 9.68357085e-07, 2.90293747e-06,
        1.94657559e-06, 2.72853896e-07, 2.13745551e-07, 1.22292732e-07,
        3.76012991e-07, 9.71559166e-06, 3.29906179e-05, 2.46234754e-06,
        8.61241369e-06, 2.97443285e-06, 1.78765515e-06, 1.33064896e-05,
        7.31496868e-07, 4.02721156e-07, 7.84514441e-07, 2.84131897e-06,
        1.52182349e-06, 7.46143132e-06, 3.54933121e-07, 7.52622284e-08,
        2.46979812e-06, 1.09635675e-05, 4.13285306e-06, 1.97714198e-05,
        1.34789582e-06, 6.64807544e-07, 1.10212613e-06, 7.281094

In [82]:
features.shape

(1, 1000)

In [83]:
tf.keras.applications.vgg16.decode_predictions(features)

[[('n03404251', 'fur_coat', 0.10946216),
  ('n04350905', 'suit', 0.10526632),
  ('n02672831', 'accordion', 0.09045777),
  ('n04536866', 'violin', 0.08386334),
  ('n04507155', 'umbrella', 0.07367158)]]

In [85]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n03404251', 'fur_coat', 0.82244325), ('n04479046', 'trench_coat', 0.036401827), ('n02667093', 'abaya', 0.020068195)]


In [87]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
import numpy as np

model = Xception(weights='imagenet')

img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n03404251', 'fur_coat', 0.68586403), ('n04479046', 'trench_coat', 0.23977722), ('n04350905', 'suit', 0.036527943)]


In [14]:
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions
import numpy as np

model = NASNetLarge(weights='imagenet')

img = image.load_img(img_path, target_size=(331, 331))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
pred = decode_predictions(preds, top=3)[0]
print('Predicted:', pred)

Predicted: [('n04479046', 'trench_coat', 0.6095649), ('n04350905', 'suit', 0.18466176), ('n03404251', 'fur_coat', 0.051378384)]


In [15]:
np.asarray(pred)[:,1:3]

array([['trench_coat', '0.6095649'],
       ['suit', '0.18466176'],
       ['fur_coat', '0.051378384']], dtype='<U11')

## Big Transfer (BiT) - m-r152x4

https://tfhub.dev/google/bit/m-r152x4/imagenet21k_classification/1

In [15]:
module = hub.KerasLayer("https://tfhub.dev/google/bit/m-r152x4/imagenet21k_classification/1")

In [16]:
image_pil = Image.open(img_path)
image = np.array(image_pil)

image = normalization_layer(image)
images = tf.expand_dims(image, axis=0)


In [17]:
logits = module(images)[0]  # Logits with shape [batch_size, 21843].
probabilities = tf.nn.sigmoid(logits)

2021-08-06 09:35:04.850063: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3126329344 exceeds 10% of free system memory.
2021-08-06 09:35:05.138799: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3133640704 exceeds 10% of free system memory.
2021-08-06 09:35:07.104146: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3126329344 exceeds 10% of free system memory.
2021-08-06 09:35:09.738582: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3126329344 exceeds 10% of free system memory.
2021-08-06 09:35:11.809843: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3126329344 exceeds 10% of free system memory.


In [31]:
labels_mapped = np.column_stack([labels_imagenet21k, probabilities[0]])

In [37]:
labels_mapped

array([['organism, being', '5.4328957e-06'],
       ['benthos', '2.6375134e-05'],
       ['heterotroph', '1.1313429e-06'],
       ...,
       ['planking', '1.4390592e-06'],
       ['chipboard, hardboard', '2.8833907e-05'],
       ['knothole', '0.00017687678']], dtype='<U175')

In [42]:
labels_mapped[:,1]

array(['5.4328957e-06', '2.6375134e-05', '1.1313429e-06', ...,
       '1.4390592e-06', '2.8833907e-05', '0.00017687678'], dtype='<U175')

In [44]:
probabilities_no_batch = probabilities[0]
np.argmax(probabilities_no_batch)

10135

In [46]:
labels_mapped[10135]

array(['scrub_brush, scrubbing_brush, scrubber', '0.0065092146'],
      dtype='<U175')

In [49]:
from keras.applications import imagenet_utils
P = imagenet_utils.decode_predictions(probabilities)

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (1, 21843)